## Example code

### Imports for the usage example

In [1]:
!pip install -r requirements.txt

  Using cached streamlit-1.9.0-py2.py3-none-any.whl (10.1 MB)
  Using cached pydeck-0.7.1-py2.py3-none-any.whl (4.3 MB)
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached blinker-1.4-py3-none-any.whl
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached validators-0.20.0-py3-none-any.whl
  Using cached pyarrow-8.0.0-cp39-cp39-win_amd64.whl (17.9 MB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached watchdog-2.1.9-py3-none-win_amd64.whl (78 kB)
  Using cached toolz-0.11.2-py3-none-any.whl (55 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached ipywidgets-7.7.1-py2.py3-none-any.whl (123 kB)
  Using cached jupyterlab_widgets-1.1.1-py3-none-any.whl (

  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached tzdata-2022.1-py2.py3-none-any.whl (339 kB)


In [1]:
from wettbewerb import load_references, save_predictions
from score import score
import os
from sklearn.model_selection import train_test_split
from train import train
from predict import predict_labels

### Get signals and split the datasets


In [2]:
ecg_leads, ecg_labels, fs, ecg_names = load_references('training')


6000	 Dateien wurden geladen.


In [3]:
ecg_train, ecg_test, labels_train, labels_test,names_train,names_test = train_test_split(ecg_leads, 
                                                                  ecg_labels,ecg_names,
                                                                  test_size=0.2, shuffle =True,stratify=ecg_labels)

### Train and save model

In [7]:
train(ecg_train,labels_train,names_train,fs,3,model_name='cnn2',epochs=60)

Epoch 1/60
2162/2162 [==============================] - 10s 4ms/step - loss: 2.1062 - accuracy: 0.4943
Epoch 2/60
2162/2162 [==============================] - 9s 4ms/step - loss: 1.5421 - accuracy: 0.6211
Epoch 3/60
2162/2162 [==============================] - 9s 4ms/step - loss: 1.3891 - accuracy: 0.6543
Epoch 4/60
2162/2162 [==============================] - 10s 5ms/step - loss: 1.3025 - accuracy: 0.6746
Epoch 5/60
2162/2162 [==============================] - 10s 4ms/step - loss: 1.2310 - accuracy: 0.6902
Epoch 6/60
2162/2162 [==============================] - 9s 4ms/step - loss: 1.1953 - accuracy: 0.6986
Epoch 7/60
2162/2162 [==============================] - 9s 4ms/step - loss: 1.1434 - accuracy: 0.7093
Epoch 8/60
2162/2162 [==============================] - 9s 4ms/step - loss: 1.1117 - accuracy: 0.7168
Epoch 9/60
2162/2162 [==============================] - 10s 4ms/step - loss: 1.0849 - accuracy: 0.7236
Epoch 10/60
2162/2162 [==============================] - 9s 4ms/step - loss: 1

### Get predictions

In [8]:
labels_predicted=predict_labels(ecg_test,fs,names_test,model_name='cnn2')

### Calculate score

In [9]:
pred_dir="cnn_1D"
pred_dir=os.path.join(os.getcwd(),pred_dir)
#os.mkdir(pred_dir)
labels_true=list()
for i,label in enumerate(labels_test):
    labels_true.append((names_test[i],label))

save_predictions(labels_predicted,pred_dir,"PREDICTIONS")
save_predictions(labels_true,pred_dir,"REFERENCE")
score(pred_dir)

1200	 Labels wurden geschrieben.
1200	 Labels wurden geschrieben.


(0.8888888888888888,
 0.607155704064201,
 {'N': {'n': 679, 'a': 16, 'o': 16, 'p': 5},
  'A': {'n': 2, 'a': 96, 'o': 6, 'p': 0},
  'O': {'n': 158, 'a': 77, 'o': 103, 'p': 5},
  'P': {'n': 14, 'a': 7, 'o': 1, 'p': 15}})

##### this 97 is only because it was tested with the train data, normaly around 85

## Full code
#### (some parts may be outdated, check train_CNN.py, predict_CNN.py and process_data.py to find the up to date functions)

In [1]:
from wettbewerb import load_references, save_predictions
from score import score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ecgdetectors import Detectors
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import statistics
from statistics import mode
from process_data import filter_ecg
from keras.models import model_from_json
import graphviz

In [5]:
model_name='Abgabe3'
json_file = open(model_name+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cnn_model = model_from_json(loaded_model_json)
# load weights into new model
cnn_model.load_weights(model_name+'.h5')
#compile the model
cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Dense(2, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [6]:
tf.keras.utils.plot_model(
    cnn_model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [3]:
ecg_leads, ecg_labels, fs, ecg_names = load_references('training')


6000	 Dateien wurden geladen.


In [4]:
#preprocess the data and rename the labels into integers

for idx, ecg_lead in enumerate(ecg_leads):
    ecg_leads[idx]=filter_ecg(ecg_leads[idx])
    ecg_leads[idx]=ecg_lead/np.amax(ecg_leads[idx])
    
for idx, label in enumerate(ecg_labels):
    if label=="N":
        ecg_labels[idx]=0
    if label=="A":
        ecg_labels[idx]=1
    if label=="O":
        ecg_labels[idx]=2
    if label=="~":
        ecg_labels[idx]=3


In [5]:
#split into training and validation datasets
ecg_train, ecg_test, labels_train, labels_test,names_train,names_test = train_test_split(ecg_leads, 
                                                                  ecg_labels,ecg_names,
                                                                  test_size=0.2, shuffle =True,stratify=ecg_labels)

### Usefull functions 
#### (some may outdated, check the process_data.py to find the up to date functions)

In [6]:
#finds the optimal value for the signal segmentation length from the traing dataset
import statistics
def find_optimal_length(hb):
    opt_len=list()
    for idx, ecg in enumerate(hb):
        opt_len.append(len(ecg))
    opt_len=statistics.median(opt_len)
    #opt_len=mode(opt_len)
    return opt_len

In [7]:
#segment the data 
import neurokit2 as nk
import pandas as pd
from ecgdetectors import Detectors
def split_HB(num_HB, ecg, labels,names,fs):
    
    #this split the data in segments of 3 heart beats
    hb=list()
    hb_labels=list()
    #this saves which split signal is part from which actual whole signal
    decoder=list()
    #start detectors as alternative
    detectors=Detectors(fs)
    for idx, ecg_lead in enumerate(ecg):
         #converts to a series
        ecg_lead=pd.Series(ecg_lead)
        #find the R peaks
        try:
            _, rpeaks = nk.ecg_peaks(ecg_lead,sampling_rate=fs)
            rpeaks=rpeaks['ECG_R_Peaks']
            prev_slice=rpeaks[0]

        #if we cant find the r peaks because sadge, then use other detector
        except:
            rpeaks = detectors.pan_tompkins_detector(ecg_lead)
            prev_slice=rpeaks[0]
            #this is the splitting of the signal
        if len(rpeaks)<2*num_HB:
            rpeaks = detectors.pan_tompkins_detector(ecg_lead)
            prev_slice=rpeaks[0]
        for count, posR in enumerate(rpeaks):
            #only extract from the 3d peak
            if (count%num_HB)==0 and len(ecg_lead[prev_slice:posR])>0:
                hb.append(ecg_lead[prev_slice:posR])
                hb_labels.append(labels[idx])
                decoder.append(names[idx])
                prev_slice=posR
    return hb, hb_labels,decoder
      

In [8]:
from imblearn.over_sampling import SMOTE 
from sklearn.decomposition import PCA
#numHB is an integer that gives the number of beats each signals will have from the original signal
def generate_data_set(numHB,ecg,labels,names,freq,smote:bool=False,minL:int=0,pca_prev=None,pca:bool=False):
    #data will be returen as a dictionary with the desired variables 
    data={}
    #first get the ecg split into hear beats
    hb,hb_labels,decoder=split_HB(numHB,ecg,labels,names,freq)
    #find the optimal length if minL was not given
    if minL<=0:
        minL=int(find_optimal_length(hb))
        
     #transforms it into a list of np array
    temp=list()
    for idx,ecg_lead in enumerate(hb): 
        temp.append(ecg_lead.to_numpy())
    #make all heart beat signals the same length, for larger signal cut them, for shorter signals just add 0s until it matches the desired length
  
    #new approach to delete the other signals that are too small
    new_ecg=list()
    new_labels=list()
    #instead of removing them, we want to do something different, add them in another list
    for idx, ecg_lead in enumerate(temp):
        if len(ecg_lead)>=minL:
            new_ecg.append(ecg_lead[:minL])
            new_labels.append(hb_labels[idx])
            #new_ecg_names.append(ecg_names[idx])
        else:
            toAdd=minL-len(ecg_lead)
            toAdd=np.zeros(toAdd)
            #merges the signal that was too short with signal full of 0 to achive the desired lenght
            new_ecg.append(np.concatenate((ecg_lead, toAdd)))
            new_labels.append(hb_labels[idx])
            #new_ecg_names.append(ecg_names[idx])
    if smote:
        sm = SMOTE()
        new_ecg, new_labels = sm.fit_resample(new_ecg, new_labels)

    
    #only use pca if we say so
    if pca:
        #if we dont send any older pca, then create its own
        if pca_prev is None:
            pca_prev=PCA(.99)
            pca_prev.fit(new_ecg)
        #use the pca to reduce the dimension of the data
        new_ecg=pca_prev.transform(new_ecg)
        data['pca']=pca_prev

 #now convert the data into working dataset for the cnn model
    x=new_ecg
    x=np.array(x)
    x=x.reshape((x.shape[0],x.shape[1],1))
    
    y=np.array(new_labels)
    
    #return the data in a dictionary way, to simplily later use of the desired variables
   
    data['input']=x
    data['labels']=y
    data['decoder']=decoder
    data['optimal_length']=minL
    
    return data

### Generate Data

In [10]:
train=generate_data_set(3,ecg_train,labels_train,names_train,fs,
                        smote=True,
                       )
#train is a dictionary, get the actual variables
x_train=train['input']
y_train=train['labels']
opt_length=train['optimal_length']
print("----------")
test=generate_data_set(3,ecg_test,labels_test,names_test,fs,
                       minL=opt_length
                      )
x_test=test['input']
y_test=test['labels']
decoder_test=test['decoder']



----------


### CNN Model

In [11]:
#THIS IS THE MODEL WE ARE USING
#CNN
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
from keras.layers import GaussianNoise
# Create sequential model 
cnn_model = tf.keras.models.Sequential()
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(5,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (x_train.shape[1:])))
cnn_model.add(Conv1D(filters=32, kernel_size=(5,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (x_train.shape[1:])))

#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), strides=1,padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Fourth CNN layer with Max pooling
cnn_model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))

#this is legit very important
cnn_model.add(GaussianNoise(0.1))
#Flatten the output

cnn_model.add(Flatten())
#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 64, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 128, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 128, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 4, activation='softmax'))

In [137]:
#OLD CNN MODEL
#CNN
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
from keras.layers import GaussianNoise
# Create sequential model 
cnn_model = tf.keras.models.Sequential()
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (x_train.shape[1:])))
#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), strides=1,padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Fourth CNN layer with Max pooling
cnn_model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))

#this is legit very important
cnn_model.add(GaussianNoise(0.1))
#Flatten the output

cnn_model.add(Flatten())
#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 64, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 128, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 4, activation='softmax'))


In [12]:
cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 715, 32)           192       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 715, 32)           5152      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 715, 64)           6208      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 715, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 358, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 358, 64)           0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 358, 64)           0

### Train model

In [13]:
epochs = 8
batch_size = 32
weights = {0: 2.,1: 10.,2: 1.,3: 1}
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode ="min", patience = 5, restore_best_weights = True)
history = cnn_model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = (x_test, y_test),
class_weight=weights
#callbacks =[earlystopping]
)

Epoch 1/8
4306/4306 [==============================] - 24s 5ms/step - loss: 1.8422 - accuracy: 0.5597 - val_loss: 1.2373 - val_accuracy: 0.5506
Epoch 2/8
4306/4306 [==============================] - 21s 5ms/step - loss: 1.2783 - accuracy: 0.6836 - val_loss: 1.1295 - val_accuracy: 0.5717
Epoch 3/8
4306/4306 [==============================] - 22s 5ms/step - loss: 1.0390 - accuracy: 0.7275 - val_loss: 1.1184 - val_accuracy: 0.6109
Epoch 4/8
4306/4306 [==============================] - 23s 5ms/step - loss: 0.9060 - accuracy: 0.7545 - val_loss: 1.0465 - val_accuracy: 0.6217
Epoch 5/8
4306/4306 [==============================] - 23s 5ms/step - loss: 0.8140 - accuracy: 0.7742 - val_loss: 1.0875 - val_accuracy: 0.6390
Epoch 6/8
4306/4306 [==============================] - 21s 5ms/step - loss: 0.7463 - accuracy: 0.7872 - val_loss: 1.0975 - val_accuracy: 0.6412
Epoch 7/8
4306/4306 [==============================] - 21s 5ms/step - loss: 0.6914 - accuracy: 0.8001 - val_loss: 1.0469 - val_accuracy:

### Get predictions and score

In [14]:
import statistics
from statistics import mode
#get the predictions
classes = cnn_model.predict(x_test)
predictions=list()
for idx, ecg_lead in enumerate (classes):
    predictions.append(np.argmax(ecg_lead))

#this dictionary will save all the predictions in groups
label_decoder={}
for idx, label in enumerate(predictions):
    if decoder_test[idx] in label_decoder:
        label_decoder[decoder_test[idx]].append(label)
    else:
        label_decoder[decoder_test[idx]]=list()
        label_decoder[decoder_test[idx]].append(label)
#to get the most frequent prediction for each signal 
labels_toTest=list()
labels_toSave=list()
for idx,keys in enumerate(label_decoder):
    #get the most frequent prediction
    frequent_label=mode(label_decoder[keys])
    #convert it into a letter
    labels2give="N"
    if int(frequent_label)==0:
        labels2give="N"
    elif int(frequent_label)==1:
        labels2give="A"
    elif int(frequent_label)==2:
        labels2give="O"
    else:
        labels2give="~"
    #save the final predictred labels
    labels_toTest.append((keys,labels2give))

for i,ecg in enumerate(labels_test):
    ecg_class=int(ecg)
    label2give="N"
    if int(ecg_class)==0:
        labels2give="N"
    elif int(ecg_class)==1:
        labels2give="A"
    elif int(ecg_class)==2:
        labels2give="O"
    else:
        labels2give="~"
    #esto es para el name
    labels_toSave.append((names_test[i],labels2give))

save_predictions(labels_toTest,model_dir,"PREDICTIONS")
save_predictions(labels_toSave,model_dir,"REFERENCE")
score(model_dir)

1200	 Labels wurden geschrieben.
1200	 Labels wurden geschrieben.


(0.8844221105527639,
 0.5709409213460968,
 {'N': {'n': 654, 'a': 7, 'o': 36, 'p': 19},
  'A': {'n': 10, 'a': 88, 'o': 2, 'p': 4},
  'O': {'n': 180, 'a': 67, 'o': 89, 'p': 7},
  'P': {'n': 12, 'a': 4, 'o': 3, 'p': 18}})